In [17]:
import pandas as pd
import numpy as np
import os
from functools import reduce

In [ ]:
print(GALAXY_INPUTS)

In [18]:
#
#study_id = GALAXY_INPUTS['study_id']


#
galaxyInput = "superkingdom"
# selection from "superkingdom" "kingdom" "phylum" "class" "order" "family" "genus" "species" "all" "only counts"
inputPath = "../data/ExampleMergeTaxonomyByRank/inputCollection/"

rankToMerge = 2     #1 = sk, 2 = k, 3 = p,...

mappingFromTaxaToNumber = {
    "superkingdom":1,
    "kingdom":2,
    "phylum":3,
    "class":4,
    "order":5,
    "family":6,
    "genus":7,
    "species":8
}

rankToMerge = mappingFromTaxaToNumber[galaxyInput]

outputName = "../data/ExampleMergeTaxonomyByRank/output"
# "Supragingival plaque.tsv"
inputFileNames = os.listdir(inputPath)


In [19]:
tables = []
for file in inputFileNames:
    # Read out all Tables and remove unneccecary informations
    df = pd.read_csv(inputPath + file,sep="\t",header=1)
    df = df.iloc[: , 1:]
    if("taxid" in df.columns):
        df = df.drop(columns=["taxid"])
    df.columns = [file.replace(".tsv",""),"#KEY"]
    columns = df.columns.tolist()
    columns.reverse()
    df = df[columns]
    tables.append(df)
# Merge all tables into one
df = reduce(lambda df1,df2:pd.merge(df1,df2,on="#KEY",how="outer"),tables)
df = df.fillna(0)
df

,#KEY,MGYA00168232-DRS020320,MGYA00168345-DRS020250,MGYA00159989-DRS049532,MGYA00160041-DRS049550,MGYA00168363-DRS020316,MGYA00168661-DRS029349,MGYA00168368-DRS020332,MGYA00168390-DRS020199,MGYA00150694-SRS675026,...,MGYA00185667-ERS2517209,MGYA00168491-DRS029411,MGYA00168312-DRS020214,MGYA00168559-DRS029310,MGYA00168307-DRS020343,MGYA00168616-DRS029351,MGYA00168256-DRS020220,MGYA00168331-DRS020268,MGYA00168488-DRS029354,MGYA00168299-DRS020213
0,sk__Bacteria,348.0,55.0,545.0,1470.0,37.0,1614.0,554.0,1022.0,48598.0,...,12709.0,977.0,633.0,1820.0,206.0,2780.0,1452.0,71.0,4308.0,631.0
1,sk__Bacteria;k__;p__Actinobacteria,2.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,28.0,...,26.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,sk__Bacteria;k__;p__Actinobacteria;c__Acidimic...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,sk__Bacteria;k__;p__Actinobacteria;c__Actinoba...,88.0,8.0,6.0,11.0,9.0,257.0,31.0,1.0,4731.0,...,53042.0,39.0,3.0,20.0,10.0,10.0,3.0,9.0,5.0,1.0
4,sk__Bacteria;k__;p__Actinobacteria;c__Actinoba...,28.0,8.0,4.0,5.0,1.0,37.0,7.0,1.0,452.0,...,244.0,15.0,4.0,5.0,4.0,13.0,1.0,10.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2170,sk__Bacteria;k__;p__Firmicutes;c__Negativicute...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2171,sk__Bacteria;k__;p__Thermotogae;c__Thermotogae...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2172,sk__Bacteria;k__;p__Actinobacteria;c__Actinoba...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2173,sk__Bacteria;k__;p__Acidobacteria;c__Solibacteres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [20]:
# group rows by taxa level
df2 = df
df2 = df2.reset_index()
for index, row in df.iterrows():
    rowRaw = row['#KEY']
    rowSplit = rowRaw.split(";")
    if(len(rowSplit) >= rankToMerge):
        df2.loc[index,"#KEY"] = rowSplit[rankToMerge-1].replace("__","_taxa_")
    else:
        df2.loc[index,"#KEY"] = "noValue"

df2 = df2[df2["#KEY"] != "noValue"]
df2 = df2.groupby(["#KEY"]).sum()
df2 = df2.reset_index()
df2 = df2.drop(columns=["index"])
df2

,#KEY,MGYA00168232-DRS020320,MGYA00168345-DRS020250,MGYA00159989-DRS049532,MGYA00160041-DRS049550,MGYA00168363-DRS020316,MGYA00168661-DRS029349,MGYA00168368-DRS020332,MGYA00168390-DRS020199,MGYA00150694-SRS675026,...,MGYA00185667-ERS2517209,MGYA00168491-DRS029411,MGYA00168312-DRS020214,MGYA00168559-DRS029310,MGYA00168307-DRS020343,MGYA00168616-DRS029351,MGYA00168256-DRS020220,MGYA00168331-DRS020268,MGYA00168488-DRS029354,MGYA00168299-DRS020213
0,sk_taxa_Archaea,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,sk_taxa_Bacteria,48783.0,20540.0,43870.0,51372.0,11036.0,128101.0,56810.0,53858.0,803947.0,...,1501422.0,104715.0,47806.0,157720.0,42753.0,152290.0,51304.0,47154.0,113641.0,47970.0
2,sk_taxa_Chloroplast,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,sk_taxa_Eukaryota,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,sk_taxa_Mitochondria,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# merge both Tables into one and save
result = pd.concat([df2,df])
result.to_csv(outputName,sep="\t",index=False)



# #KEY	anstelle von taxonomy?